In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [58]:
df1 = pd.read_csv('dataset_twitter-scraper_2023-03-27_12-06-01-847.csv')

In [59]:
df2  = pd.read_csv('2023-03-20_2023-03-27_HP PC OR HP printer.csv')

In [60]:
df1 = pd.DataFrame(df1['full_text'])

In [61]:
df2 = pd.DataFrame(df2['Text'])

In [63]:
df1.rename(columns = {'full_text' : 'Tweet'}, inplace = True)

In [64]:
df2.rename(columns = {'Text' : 'Tweet'}, inplace = True)

In [65]:
frames = [df1,df2]

In [66]:
df = pd.concat(frames)

In [67]:
df.head()

,Tweet
0,경주리모델링 \nhttps://t.co/SBZlOWDzhI
1,計算時間が短かいから多人数で使えないと\n稼働率が低くなるのだろうなぁ。\n\nビット組合わ...
2,@scubaboybop1110 STYLYさんのHPを参考に、iPhoneの動画を3DF ...
3,fyi jg aku cuma pake data internet (tsel) dan ...
4,ノートPCのキーボードとタッチパッドが壊れて使えなくなったので、VAIOからHPへ買い換えま...


In [72]:
df.shape

(964, 1)

In [73]:
df1.shape

(200, 1)

In [74]:
df2.shape

(764, 1)

# EDA

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 964 entries, 0 to 763
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   964 non-null    object
dtypes: object(1)
memory usage: 15.1+ KB


In [80]:
df.isna().sum()

Tweet    0
dtype: int64

In [82]:
from nltk.corpus import stopwords
eng_stop_words = list(stopwords.words('english'))

In [84]:
from emot.emo_unicode import UNICODE_EMOJI
emoji = list(UNICODE_EMOJI.keys())

In [92]:
import string 
import re
import textblob
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [88]:
def ProcessedTweets(text):
    text = text.lower()
    text = ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", "", text).split())
    text = re.sub(r'\@\w+|\#\w+|\d+', '', text)
    punct = str.maketrans('', '', string.punctuation+string.digits)
    text = text.translate(punct)
    tokens = word_tokenize(text)
    filtered_words = [w for w in tokens if w not in eng_stop_words]
    filtered_words = [w for w in filtered_words if w not in emoji]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = "".join(lemma_words)
    return text

In [103]:
df['Processed_Tweets'] = df['Tweet'].apply(ProcessedTweets)

In [105]:
df.tail()

,Tweet,Processed_Tweets
759,"@HPIndia hp 1005 printer,cuttack ,odisha ,yest...",hpprintercuttackodishayesterdayoneexecutivecal...
760,256MB Printer Memory for HP Laserjet Pro 400 C...,mbprintermemoryforhplaserjetprocolormdnprinter...
761,Like It👍 from Computers &amp; Accessories in A...,likeitfromcomputersampaccessoriesinamazonhpbla...
762,"https://t.co/5HWhYyjUxb ink cartridge,ink cart...",inkcartridgeinkcartridgescartridgehowtorefilli...
763,Thanks @HP for the printer that came with a ne...,thanksfortheprinterthatcamewithanearemptyprint...


In [110]:
df.drop_duplicates(subset = 'Processed_Tweets', inplace=True)

In [111]:
df.shape

(829, 2)

In [119]:
df.to_csv('final_tweet.csv')

# Sentiment Analysis

In [116]:
def polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

def sentimenttextblob(polarity):
    if polarity < -0.1:
        return "Negative"
    elif polarity > 0.1:
        return "Positive"
    else :
        return "Neutral"

In [117]:
df['Polarity'] = df['Processed_Tweets'].apply(polarity)
df['Sentiment'] = df['Polarity'].apply(sentimenttextblob)
sent = df['Sentiment'].value_counts()
sent

Neutral    829
Name: Sentiment, dtype: int64

In [123]:
tweet = "I kind of hate & love this movie!"
print(polarity(tweet)) # expected output: positive polarity


0.14166666666666664
